In [1]:
import torch
import torchvision

use_cuda = True
DEVICE = torch.device('cuda' if (use_cuda and torch.cuda.is_available()) else 'cpu')
torch.random.manual_seed(7)

In [2]:
BATCH_SIZE = 128
EPOCHS = 50
Lr = 0.01
DTYPE = torch.float32

In [3]:
transform1 = torchvision.transforms.Compose([
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.ConvertImageDtype(DTYPE),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform2 = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.ConvertImageDtype(DTYPE),
    torchvision.transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])


In [4]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform1)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform2)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class myfhNet3(torch.nn.Module):
    def __init__(self):
        super(myfhNet3, self).__init__()
        self.Conv1 = torch.nn.Conv2d(3, 8, kernel_size=11, stride=1, padding=5)
        self.Conv2 = torch.nn.Conv2d(8, 16, kernel_size=7, stride=1, padding=3)
        self.Conv3 = torch.nn.Conv2d(16, 16, kernel_size=5, stride=1, padding=2)
        self.Conv4 = torch.nn.Conv2d(16, 16, kernel_size=5, stride=1, padding=0)
        self.Conv5 = torch.nn.Conv2d(16, 16, kernel_size=5, stride=1, padding=0)
        self.Conv6 = torch.nn.Conv2d(16, 160, kernel_size=5, stride=1, padding=0)

        self.maxPool = torch.nn.MaxPool2d(kernel_size = 2, stride = 2,padding=0)

        self.fc1 = torch.nn.Linear(160*5*5, 160)
        self.fc2 = torch.nn.Linear(160, 10)

    def forward(self, x):
        y=self.Conv1(x)
        y=self.Conv2(y)
        y=self.Conv3(y)
        y=self.Conv4(y)
        y=self.Conv5(y)
        y=self.Conv6(y)

        y=self.maxPool(y)
        y=self.maxPool(y)

        y =  torch.flatten(y,1)
        y = self.fc1(y)
        y = self.fc2(y)

        return y

In [6]:
net = myfhNet3()
optimizer = torch.optim.SGD(net.parameters(), lr=Lr, momentum=0.9,weight_decay=5e-4)
loss_func = torch.nn.CrossEntropyLoss()

In [7]:
net = net.train().to(device=DEVICE)
for epoch in range(EPOCHS):  # loop over the dataset multiple times

    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs = inputs.to(device = DEVICE)
        labels = labels.to(device = DEVICE)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('[%d] loss: %.3f' % (epoch + 1, running_loss))
    running_loss = 0.0

print('Finished Training')

[1] loss: 672.447
[2] loss: 548.912
[3] loss: 493.602
[4] loss: 470.385
[5] loss: 453.028
[6] loss: 440.398
[7] loss: 433.484
[8] loss: 424.598
[9] loss: 419.596
[10] loss: 410.702
[11] loss: 404.928
[12] loss: 401.166
[13] loss: 391.219
[14] loss: 391.146
[15] loss: 387.684
[16] loss: 381.910
[17] loss: 377.559
[18] loss: 379.015
[19] loss: 373.950
[20] loss: 371.514
[21] loss: 369.665
[22] loss: 362.723
[23] loss: 364.232
[24] loss: 365.059
[25] loss: 360.779
[26] loss: 356.828
[27] loss: 357.728
[28] loss: 353.249
[29] loss: 355.825
[30] loss: 353.481
[31] loss: 348.815
[32] loss: 350.912
[33] loss: 349.717
[34] loss: 347.025
[35] loss: 346.925
[36] loss: 344.424
[37] loss: 346.020
[38] loss: 341.775
[39] loss: 340.171
[40] loss: 340.785
[41] loss: 341.559
[42] loss: 338.070
[43] loss: 334.936
[44] loss: 338.187
[45] loss: 335.001
[46] loss: 333.921
[47] loss: 335.057
[48] loss: 331.491
[49] loss: 333.464
[50] loss: 334.056
Finished Training


In [8]:
mynet=net.eval()
all_counter=0
correct_counter=0
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)
    
    out = mynet(inputs)
    out = out.detach().cpu().argmax(1)
    
    t = labels.cpu()
    for m in range(len(t)):
        all_counter += 1
        if t[m] == out[m]:
            correct_counter += 1

print(correct_counter, all_counter, correct_counter / all_counter)

6232 10000 0.6232


In [9]:
from torchsummary import summary

In [10]:
summary(net, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 32, 32]           2,912
            Conv2d-2           [-1, 16, 32, 32]           6,288
            Conv2d-3           [-1, 16, 32, 32]           6,416
            Conv2d-4           [-1, 16, 28, 28]           6,416
            Conv2d-5           [-1, 16, 24, 24]           6,416
            Conv2d-6          [-1, 160, 20, 20]          64,160
         MaxPool2d-7          [-1, 160, 10, 10]               0
         MaxPool2d-8            [-1, 160, 5, 5]               0
            Linear-9                  [-1, 160]         640,160
           Linear-10                   [-1, 10]           1,610
Total params: 734,378
Trainable params: 734,378
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 1.12
Params size (MB): 2.80
Estimated T

In [11]:
for key,value in net.state_dict().items():
    print(key,value.shape)

Conv1.weight torch.Size([8, 3, 11, 11])
Conv1.bias torch.Size([8])
Conv2.weight torch.Size([16, 8, 7, 7])
Conv2.bias torch.Size([16])
Conv3.weight torch.Size([16, 16, 5, 5])
Conv3.bias torch.Size([16])
Conv4.weight torch.Size([16, 16, 5, 5])
Conv4.bias torch.Size([16])
Conv5.weight torch.Size([16, 16, 5, 5])
Conv5.bias torch.Size([16])
Conv6.weight torch.Size([160, 16, 5, 5])
Conv6.bias torch.Size([160])
fc1.weight torch.Size([160, 4000])
fc1.bias torch.Size([160])
fc2.weight torch.Size([10, 160])
fc2.bias torch.Size([10])
